In [1]:
import agentpy as ap
import matplotlib.pyplot as plt
import IPython

In [2]:
# Road agent positions
POSITIONS = []
for i in range(0, 26):
    POSITIONS.append((13,i))
    POSITIONS.append((i,13))

In [3]:
class CleaningRoomsModel(ap.Model):
    def setup(self):
        
        # Create agents
    
        self.cars=ap.AgentList(self,2)
        self.road=ap.AgentList(self,len(POSITIONS))
        self.sign=ap.AgentList(self,2)

        # Create grid
        self.city = ap.Grid(self, [self.p.size] * 2, track_empty=True)
        
        self.city.add_agents(self.cars, [(26,13),(13,26)])
        self.city.add_agents(self.road, positions=POSITIONS ,empty=False)
        self.city.add_agents(self.sign, [(5,5),(5,20)])

        # Agent type attribute
        
        #0: car
        #1: untravelled road
        #2: travelled road
        
        #3: green light
        #4: red light
        
        self.cars.type_agent = 0
        self.road.type_agent = 1
        self.sign[0].type_agent = 3
        self.sign[1].type_agent = 4
        
        self.cars[0].type_car = 0
        self.cars[1].type_car = 1
        

    def step(self):
        cars = self.cars
        sigb = self.sign[0]
        sigc = self.sign[1]
        
        if sigb.type_agent == 3:
            sigb.type_agent = 4

        elif sigb.type_agent == 4:
            sigb.type_agent = 3
            
        if sigc.type_agent == 3:
            sigc.type_agent = 4

        elif sigc.type_agent == 4:
            sigc.type_agent = 3
        
        for car in cars:
            old_position=self.city.positions[car]
            
            for neighbor in self.city.neighbors(car):
                new_position = self.city.positions[neighbor]
                

                
                if neighbor.type_agent == 1 and sigb.type_agent == 3:
                    if car.type_car == 0:
                        self.city.move_to(car, new_position)
                        neighbor.type_agent = 2
                    break
                
                
            for neighbor in self.city.neighbors(car):
                new_position = self.city.positions[neighbor]
    
                if neighbor.type_agent == 1 and sigc.type_agent == 3:
                    if car.type_car == 1:
                        self.city.move_to(car, new_position)
                        neighbor.type_agent = 2                    
                    break

In [7]:
# Define parameters
parameters={
    'size':27,
    'steps':50,
}
# Create single-run animation with custom colors
def animation_plot (model, ax):
    attr_grid=model.city.attr_grid('type_agent')
    color_dict = {0:'#63666A', 1:'#CCB389', 2: '#D0F1EC',3:'#15b032',4:'#eb361a', None: '#FFFDFA'}
    ap.gridplot (attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Traffic simulation\n"
    f"Time-step: {model.t}")
fig, ax = plt. subplots()
model=CleaningRoomsModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))